In this experiment you will generate traffic using OSNT and capture it with tcpdump.  <img src="alt_setup2.2.jpg">
#Setup:

First time use:   
-- mkdir <crsid> (Machine B)   
-- git clone https://github.com/cucl-srg/L50 (Machine B)   
-- wget https://www.cl.cam.ac.uk/research/srg/netos/projects/netfpga/bitfiles/OSNT-SUME-live/osnt_20170129.bit -P /root/<crsid>/L50/setup (Machine B)   
-- bash /root/<crsid>/L50/setup/load_osnt_bitfile.sh <crsid> (Machine B)   
-- reboot the machine (also required after Machine B was powered down)   


Repeated use:   
-- bash /root/<crsid>/L50/setup/load_osnt_bitfile.sh <crsid> (Machine B)   
-- dagload (Machine A)  
-- bash /root/<crsid>/L50/setup/mkdir2.sh <crsid> (Machine A)   
-- eval ssh-agent -s (Machine A)   
-- ssh-add (Machine A)  
 

-- Connect nf0 (B, tx) to slf0 (A, rx) and slf1 (A, rx) via the tap.  
<img src="tap2.2_connect.jpg">

In order to listen on an interface using tcpdump, it must be up -- `ifconfig <interface> up`.

###### Usage : 

`tcpdump -i <interface> -w <output file name>` 
This runs until it is terminated. Furthermore, the capture file will not be recorded correctly unless it is killed.  
  
Eg Scripting a capture in Python
```
cmd=shlex.split('tcpdump -i eth0 -w bla.pcap')
p = Popen(cmd)
sleep(2) # ensure that process is ready to capture
# send traffic #
sleep(2) # ensure that process has finished capturing
p.terminate()
```

In [ ]:
crsid=''
%run /root/$crsid/L50/Jupyter/useful/useful.py
%run /root/$crsid/L50/Jupyter/useful/useful2.2.py
%matplotlib inline
machB_ip = '' # UPDATE THIS

The function `send` will be used throughout this Notebook:  
Traffic will be generated from NetFPGA (Machine B) using OSNT. Capture it on the NIC (Machine A) using tcpdump and check that these are correctly saved to L50Lab2/2.2.  

In [ ]:
def send(exp,num,two=False,ipg=0):

    # start capture on slf0 to L50Lab2/2.2/'+exp+'_0.erf
    cap_cmd0=shlex.split('tcpdump -i slf0 -w /root/'+crsid+'/L50Lab2/2.2/'+exp+'_0.pcap')
    p0 = Popen(cap_cmd0)
    if (two):
        # start capture on slf1 to L50Lab2/2.2/'+exp+'_1.erf
        cap_cmd1=shlex.split('tcpdump -i slf1 -w /root/'+crsid+'/L50Lab2/2.2/'+exp+'_1.pcap')
        p1 = Popen(cap_cmd1)
    sleep(2)
    
    ssh = ssh_connect(machB_ip)
    ssh_cmd('cd /root/OSNT-SUME-live/projects/osnt/sw/host/app/cli && python osnt-tool-cmd.py '
            '-ifp0 /root/'+crsid+'/L50/pcap_files/512.cap -rpn0 '+str(num)+' -ipg0 '+str(ipg)+' -run'
            ,ssh) # generate traffic
    ssh.close()
    sleep(10) # ensure that traffic has finished sending
    
    # end capture on slf0
    p0.terminate()
    local_cmd("tshark -t e -r /root/"+crsid+"/L50Lab2/2.2/"+exp+"_0.pcap | awk '{print $2}' > /root/"+crsid+"/L50Lab2/2.2/"+exp+"_0.txt")
    if (two):
        # end capture on slf1
        p1.terminate()
        local_cmd("tshark -t e -r /root/"+crsid+"/L50Lab2/2.2/"+exp+"_1.pcap | awk '{print $2}' > /root/"+crsid+"/L50Lab2/2.2/"+exp+"_1.txt")

## Experiment a

Send 1000 packets, size 512B. Check that the tool receives them. What is the precision of the measurement?

In [ ]:
%%capture
send('exp2a',1000,ipg=42560)

In [ ]:
times = gettimes('exp2a_0',crsid)
print "Packets received: " + str(len(times)) + "\n"
for i in range(10):
    print times[i][:-1]
print "..."

## Experiment b

Use the tap to send 100000 packets, size 512B, nf0 to slf0 and slf1. What is the difference between the ports?

In [ ]:
%%capture
send('exp2b',100000, two=True,ipg=42560)

The function `cmp_ports`  plots a histogram of (slf0 TS - slf1 TS).

In [ ]:
def cmp_ports(exp):
    diff = getdiff(exp,crsid)
    minn = int(min(diff) / 10.0) * 10
    maxx = int(max(diff) / 10.0) * 10
    n, bins, patches = plt.hist(diff, abs(maxx-minn)/2, (minn,maxx),log=True)
    plt.xlabel("slf0 TS - slf1 TS (microseconds)")
    plt.ylabel("number of packets")
    plt.show()
cmp_ports('exp2b')

## Experiment c

Swap the transceivers and fibres, and repeat Experiment 2.

In [ ]:
%%capture
send('exp2c',100000, two=True,ipg=42560)

In [ ]:
cmp_ports('exp2c')

## Experiment d

Send 10000 packets, size 512B, as fast as possible.  Does the tool receive them?

In [ ]:
%%capture
send('exp2d',10000)

In [ ]:
times = gettimes('exp2d_0',crsid)
print "Packets received: " + str(len(times)) + "\n"

## Experiment e

In this experiment you will ping from NIC (Machine A) to NIC (Machine B). You will capture the requests and replies on both the DAG card (Machine A) and the NIC (Machine A) using tcpdump.  <img src="alt_setup2e.jpg">
###### Setup:   
-- `rmmod ixgbe && modprobe ixgbe allow_unsupported_sfp=1` (Machine B)      
-- `dagload` (Machine A)    
-- `ifconfig slf0 192.168.0.3` (Machine A)   
-- `ifconfig intl0 192.168.0.4` (Machine B)   
-- Connect the tap as shown below. <img src="tap2e.jpg"/>

ping is used to test the reachability of a host on an IP network. The source sends an ICMP Echo Request and the destination sends an ICMP Echo Reply.

Use ping to generate 10000 RTT measurements (one at a time), and measure the RTT using DAG and tcpdump. What is the distribution? The accuracy?  
This script will capture traffic using both DAG and tcpdump.

In [ ]:
%%capture
# start DAG and tcpdump capture
cmd1=shlex.split('dagsnap -d0 -o /root/'+crsid+'/L50Lab2/2.2/exp2e_dag.erf')
p1 = Popen(cmd1)
cmd2=shlex.split("tcpdump -i slf0 -w /root/"+crsid+"/L50Lab2/2.2/exp2e_tcpdump.pcap 'icmp'")
p2 = Popen(cmd2)
sleep(2)

!ping 192.168.0.4 -i 0.005 -c 10000 -q > /root/$crsid/L50Lab2/2.2/ping.txt
sleep(2)

# end DAG and tcpdump capture
p1.kill()
p2.terminate()
!tshark -r /root/$crsid/L50Lab2/2.2/exp2e_tcpdump.pcap -V  | grep "previous captured"> \
/root/$crsid/L50Lab2/2.2/exp2e_tcpdump.txt
!dagconvert -i /root/$crsid/L50Lab2/2.2/exp2e_dag.erf -o /root/$crsid/L50Lab2/2.2/exp2e_dag_filtered.erf -b "icmp"
!tshark -r /root/$crsid/L50Lab2/2.2/exp2e_dag_filtered.erf -V  | grep "previous captured"> \
/root/$crsid/L50Lab2/2.2/exp2e_dag_filtered.txt

In [ ]:
rtt_dag = getrtt("exp2e_dag_filtered.txt",crsid)

The function `rtt_graph` takes a list of RTTs and plots a histogram.

In [ ]:
def rtt_graph(rtt):
    minn = int(floor(min(rtt) / 2.0)) * 2
    maxx = int(ceil(max(rtt) / 2.0)) * 2
    n, bins, patches = plt.hist(rtt, abs(maxx-minn)/2, (minn,maxx),log=True)
    plt.ylabel("Number of packets")
    plt.xlabel("RTT (us)")
    plt.show()
rtt_graph(rtt_dag)

In [ ]:
rtt_tdump = getrtt("exp2e_tcpdump.txt",crsid)

In [ ]:
rtt_graph(rtt_tdump)